In [224]:
# Import python modules and packages
import csv
import pandas as pd
import re

In [225]:
# Read in the raw data from the .csv file for the nursing homes and performance
nursing_df = pd.read_csv('../Resources/Provider_Info.csv')

# Verify the data was imported
print(f'Shape of DataFrame: {nursing_df.shape}')
nursing_df

Shape of DataFrame: (15640, 82)


,index,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Provider Phone Number,Provider SSA County Code,Provider County Name,...,Cycle 3 Total Health Score,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Fines,Total Amount of Fines in Dollars,Number of Payment Denials,Total Number of Penalties,Location,Processing Date
0,0,015019,MERRY WOOD LODGE CARE AND REHABILITATION CENTER,P O BOX 130,ELMORE,AL,36025,3345678484,250,Elmore,...,24.0,22.667,0,1,1,$6692.00,0,1,130 ELMORE\r\nAL 36025\r\n,09/01/2016
1,1,015113,RIVER CITY CENTER,1350 FOURTEENTH AVENUE SOUTHEAST,DECATUR,AL,35601,2563556911,510,Morgan,...,40.0,48.667,0,3,1,$6045.00,0,1,"1350 FOURTEENTH AVENUE SOUTHEAST\r\nDECATUR, A...",09/01/2016
2,2,015112,MAGNOLIA HAVEN HEALTH AND REHABILITATION CENTER,603 WRIGHT STREET,TUSKEGEE,AL,36083,3347274960,430,Macon,...,16.0,15.333,0,0,0,$0.00,0,0,"603 WRIGHT STREET\r\nTUSKEGEE, AL 36083\r\n(32...",09/01/2016
3,3,015114,SHADESCREST HEALTH CARE CENTER,331 WEST 25TH STREET,JASPER,AL,35502,2053849086,630,Walker,...,52.0,25.333,0,3,1,$189763.00,0,1,"331 WEST 25TH STREET\r\nJASPER, AL 35502\r\n(3...",09/01/2016
4,4,015123,WASHINGTON COUNTY NURSING HOME,14600 ST STEPHENS AVENUE,CHATOM,AL,36518,2518476412,640,Washington,...,20.0,13.333,0,0,0,$0.00,0,0,"14600 ST STEPHENS AVENUE\r\nCHATOM, AL 36518\r...",09/01/2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15635,15635,676372,TWIN PINES NORTH NURSING AND REHABILITATION CE...,1301 MALLETTE DRIVE,VICTORIA,TX,77904,3615769454,948,Victoria,...,NaN,9.600,1,0,0,$0.00,0,0,"1301 MALLETTE DRIVE\r\nVICTORIA, TX 77904\r\n(...",09/01/2016
15636,15636,676367,BELTERRA HEALTH & REHAB,2170 NORTH LAKE FOREST DRIVE,MCKINNEY,TX,75071,9725425500,310,Collin,...,NaN,24.000,0,0,0,$0.00,0,0,"2170 NORTH LAKE FOREST DRIVE\r\nMCKINNEY, TX 7...",09/01/2016
15637,15637,676391,WINDSOR CALALLEN,4162 WILDCAT DR,CORPUS CHRISTI,TX,78410,3612412954,830,Nueces,...,NaN,NaN,0,0,0,$0.00,0,0,"4162 WILDCAT DR\r\nCORPUS CHRISTI, TX 78410\r\...",09/01/2016
15638,15638,676377,RELIANT DALLAS SUB-ACUTE,7930 NORTHAVEN ROAD,DALLAS,TX,75230,2147068230,390,Dallas,...,NaN,NaN,0,0,0,$0.00,0,0,"7930 NORTHAVEN ROAD\r\nDALLAS, TX 75230\r\n(32...",09/01/2016


In [226]:
# Testing for duplicate rows
print(f'Initial number of rows: {nursing_df.shape[0]}')
nursing_df = nursing_df.drop_duplicates()
print(f'Number of rows after dropping duplicate rows: {nursing_df.shape[0]}')

Initial number of rows: 15640
Number of rows after dropping duplicate rows: 15640


In [227]:
# Testing for uniqueness of 'Federal Provider Number'
print(f'Number of rows: {nursing_df.shape[0]}')
print(f'Number of unique "Federal Provider Number"s: {len(nursing_df["Federal Provider Number"].unique())}')

Number of rows: 15640
Number of unique "Federal Provider Number"s: 15640


In [228]:
# Because the the resulting numbers are the same in the cell above, we can conclude that there are no 
# duplicate "Federal Provider Number"s and so it can be used as a primary key. 

In [229]:
# Show all the columns
nursing_df.columns

Index(['index', 'Federal Provider Number', 'Provider Name', 'Provider Address',
       'Provider City', 'Provider State', 'Provider Zip Code',
       'Provider Phone Number', 'Provider SSA County Code',
       'Provider County Name', 'Ownership Type', 'Number of Certified Beds',
       'Number of Residents in Certified Beds', 'Provider Type',
       'Provider Resides in Hospital', 'Legal Business Name',
       'Date First Approved to Provide Medicare and Medicaid services',
       'Continuing Care Retirement Community', 'Special Focus Facility',
       'Most Recent Health Inspection More Than 2 Years Ago',
       'Provider Changed Ownership in Last 12 Months',
       'With a Resident and Family Council',
       'Automatic Sprinkler Systems in All Required Areas', 'Overall Rating',
       'Overall Rating Footnote', 'Health Inspection Rating',
       'Health Inspection Rating Footnote', 'QM Rating', 'QM Rating Footnote',
       'Staffing Rating', 'Staffing Rating Footnote', 'RN Staffing 

In [230]:
# Investigate the types of the columns that are present
nursing_df.dtypes

index                                int64
Federal Provider Number             object
Provider Name                       object
Provider Address                    object
Provider City                       object
                                     ...  
Total Amount of Fines in Dollars    object
Number of Payment Denials            int64
Total Number of Penalties            int64
Location                            object
Processing Date                     object
Length: 82, dtype: object

In [231]:
# Investigate the count of non-nan values for the columns
print(nursing_df.count().loc[nursing_df.count() < nursing_df.shape[0]])

Overall Rating                                                     15505
Overall Rating Footnote                                              135
Health Inspection Rating                                           15505
Health Inspection Rating Footnote                                    135
QM Rating                                                          15477
QM Rating Footnote                                                   163
Staffing Rating                                                    15198
Staffing Rating Footnote                                             442
RN Staffing Rating                                                 15198
RN Staffing Rating Footnote                                          442
Reported Staffing Footnote                                           396
Physical Therapist Staffing Footnote                                 398
Reported CNA Staffing Hours per Resident per Day                   15244
Reported LPN Staffing Hours per Resident per Day   

In [232]:
# Because location plays an important role in decision making for all sorts of reasons
# (e.g. weather, taxes, large city or rural area, proximity to family, ... etc.), any 
# nursing home without a location associated with it will be dropped from consideration
no_nan_n_df = nursing_df.dropna(subset = ['Location'])
no_nan_n_df.count()

index                               15613
Federal Provider Number             15613
Provider Name                       15613
Provider Address                    15613
Provider City                       15613
                                    ...  
Total Amount of Fines in Dollars    15613
Number of Payment Denials           15613
Total Number of Penalties           15613
Location                            15613
Processing Date                     15613
Length: 82, dtype: int64

In [233]:
# Extract coordinates from the 'Location' column and insert 'Null' if there are no
# coordinates listed for a location.
coord_n_df = no_nan_n_df.copy()
coord_n_df['Coordinates'] = [re.search('\([-]?\d+\.\d+, [-]?\d+\.\d+\)', row).group() if len(re.findall('\([-]?\d+\.\d+, [-]?\d+\.\d+\)', row)) > 0 else 'Null' for row in no_nan_n_df['Location'] ]

In [234]:
# Verify that the column Coordinates was added
print(coord_n_df.shape)
coord_n_df[['Federal Provider Number', 'Location', 'Coordinates']]

(15613, 83)


,Federal Provider Number,Location,Coordinates
0,015019,130 ELMORE\r\nAL 36025\r\n,Null
1,015113,"1350 FOURTEENTH AVENUE SOUTHEAST\r\nDECATUR, A...","(34.58636, -86.970399)"
2,015112,"603 WRIGHT STREET\r\nTUSKEGEE, AL 36083\r\n(32...","(32.432923, -85.691051)"
3,015114,"331 WEST 25TH STREET\r\nJASPER, AL 35502\r\n(3...","(33.826573, -87.279027)"
4,015123,"14600 ST STEPHENS AVENUE\r\nCHATOM, AL 36518\r...","(31.460032, -88.234524)"
...,...,...,...
15635,676372,"1301 MALLETTE DRIVE\r\nVICTORIA, TX 77904\r\n(...","(28.869706, -97.011339)"
15636,676367,"2170 NORTH LAKE FOREST DRIVE\r\nMCKINNEY, TX 7...","(33.218111, -96.679208)"
15637,676391,"4162 WILDCAT DR\r\nCORPUS CHRISTI, TX 78410\r\...","(27.858891, -97.633984)"
15638,676377,"7930 NORTHAVEN ROAD\r\nDALLAS, TX 75230\r\n(32...","(32.901603, -96.770271)"


In [235]:

# Remove all rows that have a 'Null' for their 'Coordinates' value primarily b/c
# the time comsumed for tracking down why coordinates couldn't be determined is beyond
# the limits of this project's time horizon.
coord_n_df = coord_n_df.loc[coord_n_df['Coordinates'] != 'Null']

#Verify the any 'Null' Location rows were removed
print(coord_n_df.shape)
coord_n_df.reset_index(inplace=True)
coord_n_df.head(10)

(15119, 83)


,level_0,index,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Provider Phone Number,Provider SSA County Code,...,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Fines,Total Amount of Fines in Dollars,Number of Payment Denials,Total Number of Penalties,Location,Processing Date,Coordinates
0,1,1,015113,RIVER CITY CENTER,1350 FOURTEENTH AVENUE SOUTHEAST,DECATUR,AL,35601,2563556911,510,...,48.667,0,3,1,$6045.00,0,1,"1350 FOURTEENTH AVENUE SOUTHEAST\r\nDECATUR, A...",09/01/2016,"(34.58636, -86.970399)"
1,2,2,015112,MAGNOLIA HAVEN HEALTH AND REHABILITATION CENTER,603 WRIGHT STREET,TUSKEGEE,AL,36083,3347274960,430,...,15.333,0,0,0,$0.00,0,0,"603 WRIGHT STREET\r\nTUSKEGEE, AL 36083\r\n(32...",09/01/2016,"(32.432923, -85.691051)"
2,3,3,015114,SHADESCREST HEALTH CARE CENTER,331 WEST 25TH STREET,JASPER,AL,35502,2053849086,630,...,25.333,0,3,1,$189763.00,0,1,"331 WEST 25TH STREET\r\nJASPER, AL 35502\r\n(3...",09/01/2016,"(33.826573, -87.279027)"
3,4,4,015123,WASHINGTON COUNTY NURSING HOME,14600 ST STEPHENS AVENUE,CHATOM,AL,36518,2518476412,640,...,13.333,0,0,0,$0.00,0,0,"14600 ST STEPHENS AVENUE\r\nCHATOM, AL 36518\r...",09/01/2016,"(31.460032, -88.234524)"
4,5,5,015115,"CORDOVA HEALTH AND REHABILITATION, LLC",70 HIGHLAND STREET WEST,CORDOVA,AL,35550,2054839282,630,...,32.667,0,0,0,$0.00,0,0,"70 HIGHLAND STREET WEST\r\nCORDOVA, AL 35550\r...",09/01/2016,"(33.762149, -87.184514)"
5,6,6,015116,SIGNATURE HEALTHCARE OF WHITESBURG GARDENS,105 TEAKWOOD DRIVE SW,HUNTSVILLE,AL,35801,2568815000,440,...,52.000,0,1,0,$0.00,0,0,"105 TEAKWOOD DRIVE SW\r\nHUNTSVILLE, AL 35801\...",09/01/2016,"(34.697369, -86.570383)"
6,7,7,015138,HENDRIX HEALTH AND REHABILITATION,1000 HIGHWAY 33,DOUBLE SPRINGS,AL,35553,2054892136,660,...,14.667,0,0,0,$0.00,0,0,"1000 HIGHWAY\r\nDOUBLE SPRINGS, AL 35553\r\n(3...",09/01/2016,"(34.163162, -87.409783)"
7,8,8,015137,ALICEVILLE MANOR NURSING HOME,703 17TH STREET NORTHWEST,ALICEVILLE,AL,35442,2053736307,530,...,5.333,0,0,0,$0.00,0,0,"703 17TH STREET NORTHWEST\r\nALICEVILLE, AL 35...",09/01/2016,"(33.134382, -88.16865)"
8,9,9,015136,FALKVILLE HEALTH CARE CENTER,10 WEST 3RD STREET PO BOX 409,FALKVILLE,AL,35622,2567845291,510,...,23.333,0,1,0,$0.00,0,0,"10 WEST 3RD STREET\r\nFALKVILLE, AL 35622\r\n(...",09/01/2016,"(34.368669, -86.910173)"
9,10,10,015151,AZALEA GARDENS OF MOBILE,1758 SPRINGHILL AVE,MOBILE,AL,36607,2514790551,480,...,44.667,0,2,1,$6500.00,0,1,"1758 SPRINGHILL AVE\r\nMOBILE, AL 36607\r\n(30...",09/01/2016,"(30.691974, -88.082071)"


In [236]:

# Remove extra column created by reset_index
del coord_n_df['level_0']
coord_n_df

,index,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Provider Phone Number,Provider SSA County Code,Provider County Name,...,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Fines,Total Amount of Fines in Dollars,Number of Payment Denials,Total Number of Penalties,Location,Processing Date,Coordinates
0,1,015113,RIVER CITY CENTER,1350 FOURTEENTH AVENUE SOUTHEAST,DECATUR,AL,35601,2563556911,510,Morgan,...,48.667,0,3,1,$6045.00,0,1,"1350 FOURTEENTH AVENUE SOUTHEAST\r\nDECATUR, A...",09/01/2016,"(34.58636, -86.970399)"
1,2,015112,MAGNOLIA HAVEN HEALTH AND REHABILITATION CENTER,603 WRIGHT STREET,TUSKEGEE,AL,36083,3347274960,430,Macon,...,15.333,0,0,0,$0.00,0,0,"603 WRIGHT STREET\r\nTUSKEGEE, AL 36083\r\n(32...",09/01/2016,"(32.432923, -85.691051)"
2,3,015114,SHADESCREST HEALTH CARE CENTER,331 WEST 25TH STREET,JASPER,AL,35502,2053849086,630,Walker,...,25.333,0,3,1,$189763.00,0,1,"331 WEST 25TH STREET\r\nJASPER, AL 35502\r\n(3...",09/01/2016,"(33.826573, -87.279027)"
3,4,015123,WASHINGTON COUNTY NURSING HOME,14600 ST STEPHENS AVENUE,CHATOM,AL,36518,2518476412,640,Washington,...,13.333,0,0,0,$0.00,0,0,"14600 ST STEPHENS AVENUE\r\nCHATOM, AL 36518\r...",09/01/2016,"(31.460032, -88.234524)"
4,5,015115,"CORDOVA HEALTH AND REHABILITATION, LLC",70 HIGHLAND STREET WEST,CORDOVA,AL,35550,2054839282,630,Walker,...,32.667,0,0,0,$0.00,0,0,"70 HIGHLAND STREET WEST\r\nCORDOVA, AL 35550\r...",09/01/2016,"(33.762149, -87.184514)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15114,15635,676372,TWIN PINES NORTH NURSING AND REHABILITATION CE...,1301 MALLETTE DRIVE,VICTORIA,TX,77904,3615769454,948,Victoria,...,9.600,1,0,0,$0.00,0,0,"1301 MALLETTE DRIVE\r\nVICTORIA, TX 77904\r\n(...",09/01/2016,"(28.869706, -97.011339)"
15115,15636,676367,BELTERRA HEALTH & REHAB,2170 NORTH LAKE FOREST DRIVE,MCKINNEY,TX,75071,9725425500,310,Collin,...,24.000,0,0,0,$0.00,0,0,"2170 NORTH LAKE FOREST DRIVE\r\nMCKINNEY, TX 7...",09/01/2016,"(33.218111, -96.679208)"
15116,15637,676391,WINDSOR CALALLEN,4162 WILDCAT DR,CORPUS CHRISTI,TX,78410,3612412954,830,Nueces,...,NaN,0,0,0,$0.00,0,0,"4162 WILDCAT DR\r\nCORPUS CHRISTI, TX 78410\r\...",09/01/2016,"(27.858891, -97.633984)"
15117,15638,676377,RELIANT DALLAS SUB-ACUTE,7930 NORTHAVEN ROAD,DALLAS,TX,75230,2147068230,390,Dallas,...,NaN,0,0,0,$0.00,0,0,"7930 NORTHAVEN ROAD\r\nDALLAS, TX 75230\r\n(32...",09/01/2016,"(32.901603, -96.770271)"


In [237]:
# Extract the Latitude from the Coordinates column and create a Latitude column with the value
# extracted
coord_n_df['Latitude'] = [re.search('[-]?\d+\.\d+', row).group() for row in coord_n_df['Coordinates']]
coord_n_df.head()
coord_n_df[['Federal Provider Number', 'Location', 'Coordinates', 'Latitude']]

,Federal Provider Number,Location,Coordinates,Latitude
0,015113,"1350 FOURTEENTH AVENUE SOUTHEAST\r\nDECATUR, A...","(34.58636, -86.970399)",34.58636
1,015112,"603 WRIGHT STREET\r\nTUSKEGEE, AL 36083\r\n(32...","(32.432923, -85.691051)",32.432923
2,015114,"331 WEST 25TH STREET\r\nJASPER, AL 35502\r\n(3...","(33.826573, -87.279027)",33.826573
3,015123,"14600 ST STEPHENS AVENUE\r\nCHATOM, AL 36518\r...","(31.460032, -88.234524)",31.460032
4,015115,"70 HIGHLAND STREET WEST\r\nCORDOVA, AL 35550\r...","(33.762149, -87.184514)",33.762149
...,...,...,...,...
15114,676372,"1301 MALLETTE DRIVE\r\nVICTORIA, TX 77904\r\n(...","(28.869706, -97.011339)",28.869706
15115,676367,"2170 NORTH LAKE FOREST DRIVE\r\nMCKINNEY, TX 7...","(33.218111, -96.679208)",33.218111
15116,676391,"4162 WILDCAT DR\r\nCORPUS CHRISTI, TX 78410\r\...","(27.858891, -97.633984)",27.858891
15117,676377,"7930 NORTHAVEN ROAD\r\nDALLAS, TX 75230\r\n(32...","(32.901603, -96.770271)",32.901603


In [238]:
# Extract the Longitude from the Coordinates column and create a Longitude column with the value
# extracted
coord_n_df['Longitude'] = [re.findall('[-]?\d+\.\d+', row)[1] for row in coord_n_df['Coordinates']]
print(coord_n_df.shape)
coord_n_df[['Federal Provider Number', 'Location', 'Coordinates', 'Latitude', 'Longitude']]

(15119, 85)


,Federal Provider Number,Location,Coordinates,Latitude,Longitude
0,015113,"1350 FOURTEENTH AVENUE SOUTHEAST\r\nDECATUR, A...","(34.58636, -86.970399)",34.58636,-86.970399
1,015112,"603 WRIGHT STREET\r\nTUSKEGEE, AL 36083\r\n(32...","(32.432923, -85.691051)",32.432923,-85.691051
2,015114,"331 WEST 25TH STREET\r\nJASPER, AL 35502\r\n(3...","(33.826573, -87.279027)",33.826573,-87.279027
3,015123,"14600 ST STEPHENS AVENUE\r\nCHATOM, AL 36518\r...","(31.460032, -88.234524)",31.460032,-88.234524
4,015115,"70 HIGHLAND STREET WEST\r\nCORDOVA, AL 35550\r...","(33.762149, -87.184514)",33.762149,-87.184514
...,...,...,...,...,...
15114,676372,"1301 MALLETTE DRIVE\r\nVICTORIA, TX 77904\r\n(...","(28.869706, -97.011339)",28.869706,-97.011339
15115,676367,"2170 NORTH LAKE FOREST DRIVE\r\nMCKINNEY, TX 7...","(33.218111, -96.679208)",33.218111,-96.679208
15116,676391,"4162 WILDCAT DR\r\nCORPUS CHRISTI, TX 78410\r\...","(27.858891, -97.633984)",27.858891,-97.633984
15117,676377,"7930 NORTHAVEN ROAD\r\nDALLAS, TX 75230\r\n(32...","(32.901603, -96.770271)",32.901603,-96.770271


In [239]:
# State the number of columns before deleting a column, so as to verify the
# column deletion after the fact.
len(coord_n_df.columns)

85

In [240]:
# Deleting the redundant Coordinates column (Latitude and Longitude columns
# make it redundant).
del coord_n_df['Coordinates']

In [241]:

# Verify that the column count reduced by 1
len(coord_n_df.columns)

84

In [242]:

# Verify that the Coordinates column is gone and the rest of the dataframe is 
# unaffected
coord_n_df.head()

,index,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Provider Phone Number,Provider SSA County Code,Provider County Name,...,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Fines,Total Amount of Fines in Dollars,Number of Payment Denials,Total Number of Penalties,Location,Processing Date,Latitude,Longitude
0,1,015113,RIVER CITY CENTER,1350 FOURTEENTH AVENUE SOUTHEAST,DECATUR,AL,35601,2563556911,510,Morgan,...,0,3,1,$6045.00,0,1,"1350 FOURTEENTH AVENUE SOUTHEAST\r\nDECATUR, A...",09/01/2016,34.58636,-86.970399
1,2,015112,MAGNOLIA HAVEN HEALTH AND REHABILITATION CENTER,603 WRIGHT STREET,TUSKEGEE,AL,36083,3347274960,430,Macon,...,0,0,0,$0.00,0,0,"603 WRIGHT STREET\r\nTUSKEGEE, AL 36083\r\n(32...",09/01/2016,32.432923,-85.691051
2,3,015114,SHADESCREST HEALTH CARE CENTER,331 WEST 25TH STREET,JASPER,AL,35502,2053849086,630,Walker,...,0,3,1,$189763.00,0,1,"331 WEST 25TH STREET\r\nJASPER, AL 35502\r\n(3...",09/01/2016,33.826573,-87.279027
3,4,015123,WASHINGTON COUNTY NURSING HOME,14600 ST STEPHENS AVENUE,CHATOM,AL,36518,2518476412,640,Washington,...,0,0,0,$0.00,0,0,"14600 ST STEPHENS AVENUE\r\nCHATOM, AL 36518\r...",09/01/2016,31.460032,-88.234524
4,5,015115,"CORDOVA HEALTH AND REHABILITATION, LLC",70 HIGHLAND STREET WEST,CORDOVA,AL,35550,2054839282,630,Walker,...,0,0,0,$0.00,0,0,"70 HIGHLAND STREET WEST\r\nCORDOVA, AL 35550\r...",09/01/2016,33.762149,-87.184514


In [243]:
# Print all columns as an aid in deciding which columns to keep and which to
# truncate from the dataframe
print(len(coord_n_df.columns))
coord_n_df.columns


84


Index(['index', 'Federal Provider Number', 'Provider Name', 'Provider Address',
       'Provider City', 'Provider State', 'Provider Zip Code',
       'Provider Phone Number', 'Provider SSA County Code',
       'Provider County Name', 'Ownership Type', 'Number of Certified Beds',
       'Number of Residents in Certified Beds', 'Provider Type',
       'Provider Resides in Hospital', 'Legal Business Name',
       'Date First Approved to Provide Medicare and Medicaid services',
       'Continuing Care Retirement Community', 'Special Focus Facility',
       'Most Recent Health Inspection More Than 2 Years Ago',
       'Provider Changed Ownership in Last 12 Months',
       'With a Resident and Family Council',
       'Automatic Sprinkler Systems in All Required Areas', 'Overall Rating',
       'Overall Rating Footnote', 'Health Inspection Rating',
       'Health Inspection Rating Footnote', 'QM Rating', 'QM Rating Footnote',
       'Staffing Rating', 'Staffing Rating Footnote', 'RN Staffing 

In [244]:
# Remove extraneous columns
list_of_columns_to_keep = ['Federal Provider Number', 'Provider Name', 
    'Provider City', 'Provider State', 'Provider Zip Code', 'Provider County Name',
    'Ownership Type', 'Number of Certified Beds', 'Number of Residents in Certified Beds', 
    'Provider Type', 'Provider Resides in Hospital', 
    'Most Recent Health Inspection More Than 2 Years Ago', 
    'Automatic Sprinkler Systems in All Required Areas', 'Overall Rating',
    'Health Inspection Rating','Staffing Rating','RN Staffing Rating',
    'Total Weighted Health Survey Score','Number of Facility Reported Incidents',
    'Number of Substantiated Complaints', 'Number of Fines',
    'Total Amount of Fines in Dollars', 'Number of Payment Denials',
    'Total Number of Penalties', 'Location', 'Processing Date', 'Latitude',
    'Adjusted Total Nurse Staffing Hours per Resident per Day', 'Longitude']

truncated_nursing_df = pd.DataFrame()
truncated_nursing_df = coord_n_df[list_of_columns_to_keep]


In [245]:
# Assure that the columns to keep were kept and the columns to remove
# were removed
print(len(truncated_nursing_df.columns))
truncated_nursing_df.columns

29


Index(['Federal Provider Number', 'Provider Name', 'Provider City',
       'Provider State', 'Provider Zip Code', 'Provider County Name',
       'Ownership Type', 'Number of Certified Beds',
       'Number of Residents in Certified Beds', 'Provider Type',
       'Provider Resides in Hospital',
       'Most Recent Health Inspection More Than 2 Years Ago',
       'Automatic Sprinkler Systems in All Required Areas', 'Overall Rating',
       'Health Inspection Rating', 'Staffing Rating', 'RN Staffing Rating',
       'Total Weighted Health Survey Score',
       'Number of Facility Reported Incidents',
       'Number of Substantiated Complaints', 'Number of Fines',
       'Total Amount of Fines in Dollars', 'Number of Payment Denials',
       'Total Number of Penalties', 'Location', 'Processing Date', 'Latitude',
       'Adjusted Total Nurse Staffing Hours per Resident per Day',
       'Longitude'],
      dtype='object')

In [246]:
# Calculating R Coefficients between columns
correlations_df = truncated_nursing_df.corr()
correlations_df

C:\Users\T852\AppData\Local\Temp\ipykernel_216972\3664722020.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlations_df = truncated_nursing_df.corr()


,Provider Zip Code,Number of Certified Beds,Number of Residents in Certified Beds,Provider Resides in Hospital,Most Recent Health Inspection More Than 2 Years Ago,Overall Rating,Health Inspection Rating,Staffing Rating,RN Staffing Rating,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Fines,Number of Payment Denials,Total Number of Penalties,Adjusted Total Nurse Staffing Hours per Resident per Day
Provider Zip Code,1.000000,-0.175971,-0.246064,0.022419,0.000004,0.011350,-0.001988,0.019494,-0.065033,0.198754,0.132162,0.084715,0.038334,0.114800,0.073342,0.092353
Number of Certified Beds,-0.175971,1.000000,0.923585,-0.148536,-0.026011,-0.177076,-0.169983,-0.206938,-0.205187,0.078245,0.091545,0.209582,0.075043,0.029804,0.071583,-0.196783
Number of Residents in Certified Beds,-0.246064,0.923585,1.000000,-0.140992,-0.024258,-0.136609,-0.133849,-0.205556,-0.198600,0.028913,0.082893,0.157475,0.043969,-0.011505,0.031244,-0.205458
Provider Resides in Hospital,0.022419,-0.148536,-0.140992,1.000000,0.015032,0.101353,0.103410,0.188538,0.161519,-0.048840,-0.025148,-0.099568,-0.046814,-0.018318,-0.044554,0.242003
Most Recent Health Inspection More Than 2 Years Ago,0.000004,-0.026011,-0.024258,0.015032,1.000000,0.000242,-0.004956,0.018877,0.020911,0.020017,-0.008041,-0.009028,-0.009638,-0.000552,-0.007984,0.014354
Overall Rating,0.011350,-0.177076,-0.136609,0.101353,0.000242,1.000000,0.839331,0.466591,0.426148,-0.552923,-0.174330,-0.382881,-0.350939,-0.259359,-0.379139,0.324305
Health Inspection Rating,-0.001988,-0.169983,-0.133849,0.103410,-0.004956,0.839331,1.000000,0.183518,0.189953,-0.614811,-0.225469,-0.413672,-0.389060,-0.274452,-0.415490,0.158568
Staffing Rating,0.019494,-0.206938,-0.205556,0.188538,0.018877,0.466591,0.183518,1.000000,0.797777,-0.144152,0.015987,-0.148361,-0.067926,-0.055061,-0.075202,0.693626
RN Staffing Rating,-0.065033,-0.205187,-0.198600,0.161519,0.020911,0.426148,0.189953,0.797777,1.000000,-0.172038,0.011355,-0.132248,-0.083301,-0.071549,-0.093710,0.419558
Total Weighted Health Survey Score,0.198754,0.078245,0.028913,-0.048840,0.020017,-0.552923,-0.614811,-0.144152,-0.172038,1.000000,0.267858,0.418180,0.409955,0.469694,0.504506,-0.080101


In [247]:
correlations_df.insert(0, 'Column Of Category', correlations_df.index)
correlations_df

,Column Of Category,Provider Zip Code,Number of Certified Beds,Number of Residents in Certified Beds,Provider Resides in Hospital,Most Recent Health Inspection More Than 2 Years Ago,Overall Rating,Health Inspection Rating,Staffing Rating,RN Staffing Rating,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Fines,Number of Payment Denials,Total Number of Penalties,Adjusted Total Nurse Staffing Hours per Resident per Day
Provider Zip Code,Provider Zip Code,1.000000,-0.175971,-0.246064,0.022419,0.000004,0.011350,-0.001988,0.019494,-0.065033,0.198754,0.132162,0.084715,0.038334,0.114800,0.073342,0.092353
Number of Certified Beds,Number of Certified Beds,-0.175971,1.000000,0.923585,-0.148536,-0.026011,-0.177076,-0.169983,-0.206938,-0.205187,0.078245,0.091545,0.209582,0.075043,0.029804,0.071583,-0.196783
Number of Residents in Certified Beds,Number of Residents in Certified Beds,-0.246064,0.923585,1.000000,-0.140992,-0.024258,-0.136609,-0.133849,-0.205556,-0.198600,0.028913,0.082893,0.157475,0.043969,-0.011505,0.031244,-0.205458
Provider Resides in Hospital,Provider Resides in Hospital,0.022419,-0.148536,-0.140992,1.000000,0.015032,0.101353,0.103410,0.188538,0.161519,-0.048840,-0.025148,-0.099568,-0.046814,-0.018318,-0.044554,0.242003
Most Recent Health Inspection More Than 2 Years Ago,Most Recent Health Inspection More Than 2 Year...,0.000004,-0.026011,-0.024258,0.015032,1.000000,0.000242,-0.004956,0.018877,0.020911,0.020017,-0.008041,-0.009028,-0.009638,-0.000552,-0.007984,0.014354
Overall Rating,Overall Rating,0.011350,-0.177076,-0.136609,0.101353,0.000242,1.000000,0.839331,0.466591,0.426148,-0.552923,-0.174330,-0.382881,-0.350939,-0.259359,-0.379139,0.324305
Health Inspection Rating,Health Inspection Rating,-0.001988,-0.169983,-0.133849,0.103410,-0.004956,0.839331,1.000000,0.183518,0.189953,-0.614811,-0.225469,-0.413672,-0.389060,-0.274452,-0.415490,0.158568
Staffing Rating,Staffing Rating,0.019494,-0.206938,-0.205556,0.188538,0.018877,0.466591,0.183518,1.000000,0.797777,-0.144152,0.015987,-0.148361,-0.067926,-0.055061,-0.075202,0.693626
RN Staffing Rating,RN Staffing Rating,-0.065033,-0.205187,-0.198600,0.161519,0.020911,0.426148,0.189953,0.797777,1.000000,-0.172038,0.011355,-0.132248,-0.083301,-0.071549,-0.093710,0.419558
Total Weighted Health Survey Score,Total Weighted Health Survey Score,0.198754,0.078245,0.028913,-0.048840,0.020017,-0.552923,-0.614811,-0.144152,-0.172038,1.000000,0.267858,0.418180,0.409955,0.469694,0.504506,-0.080101


In [248]:

# Save correlations to file
correlations_df.to_json('./correlations_df_by_record.json', orient='records')

In [ ]:
# Create a saving point (for the loading point in the cell below)
truncated_nursing_df.to_json('./truncated_nursing_df1.json')
truncated_nursing_df.to_json('./truncated_nursing_df1_by_record.json', orient='records')
truncated_nursing_df.to_csv('./truncated_nursing_df1.csv', index=False)

In [ ]:
# Create a loading point to skip all the previous cells when rerunning
# the code further past this cell
truncated_nursing_df = pd.read_csv('./truncated_nursing_df1.csv')

In [ ]:
# Verify that the load of the data was successful
print(truncated_nursing_df.shape)
truncated_nursing_df.head()

In [ ]:
# Create the a performance table
list_of_performance_columns_df = ['Most Recent Health Inspection More Than 2 Years Ago', 'Overall Rating',
       'Health Inspection Rating', 'Staffing Rating', 'RN Staffing Rating',
       'Total Weighted Health Survey Score',
       'Number of Facility Reported Incidents',
       'Number of Substantiated Complaints', 'Number of Fines',
       'Total Amount of Fines in Dollars', 'Number of Payment Denials',
       'Total Number of Penalties']

performance_table_df = pd.DataFrame()
performance_table_df[list_of_performance_columns_df] = truncated_nursing_df[list_of_performance_columns_df]

# Verify that the performance table was created
print(performance_table_df.shape)
performance_table_df.head()

In [ ]:
# Show the number of rows in performance_table_df just before any duplicate rows are
# dropped to verify that rows were dropped.
performance_table_df.shape

In [ ]:
# Remove dupulicate rows
performance_table_df.drop_duplicates(subset=list_of_performance_columns_df, inplace=True)

In [ ]:
# Verify that rows were dropped.
print(performance_table_df.shape)

In [ ]:
# Create a primary key for performance_table_df called 'performance_id'. 
performance_table_df['performance_id'] = [i for i in range(performance_table_df.shape[0])]

# Verify that 'performance_id' was created
print(performance_table_df.shape)
performance_table_df.head()

In [ ]:
# Save performance table data to a .csv and .json (for the loading point in the cell below and 
# for data manipulations, respectively)
performance_table_df.to_csv('../TableData/performance_table.csv', index=False)
performance_table_df.to_json('../TableData/performance_table.json')

In [ ]:
# Create a loading point to skip all the previous cells when rerunning
# the code further past this cell
performance_table_df = pd.read_csv('../TableData/performance_table.csv')

In [ ]:
# Verify that the load of the data was successful
print(performance_table_df.shape)
performance_table_df.head()

In [ ]:
# Show the shape of truncated_nursing_df just before adding a column for verification
# after the column is added
truncated_nursing_df.shape

In [ ]:
# Add a column for the foreign key called performance_id in what will be used
# to create the business_table_df further below and initialize performance_id
# to an unused and recognizable value (-1)
truncated_nursing_df['performance_id'] = -1

In [ ]:

# Verify that the column was added
print(truncated_nursing_df.shape)
truncated_nursing_df[['Provider Name', 'performance_id']]

In [ ]:
# Add corrected performance_id foriegn key to truncated_nursing_df
for i in range(truncated_nursing_df.shape[0]):
    for j in range(performance_table_df.shape[0]):
        all_performance_columns_match = True
        for k in range(len(list_of_performance_columns_df)):

            # Must convert the value to str otherwise the 'nan' of truncated_nursing_df can't
            # match the 'nan' of performance_table_df
            if str(truncated_nursing_df[list_of_performance_columns_df[k]].iloc[i]) != str(performance_table_df[list_of_performance_columns_df[k]].iloc[j]):
                all_performance_columns_match = False
                break
        if  all_performance_columns_match:     
            truncated_nursing_df['performance_id'].iloc[i] = performance_table_df['performance_id'].iloc[j]
            break



In [ ]:
# Verify that foreign keys 'perfomance_id' are no longer '-1'
print(len(truncated_nursing_df[truncated_nursing_df['performance_id'] != '-1']))
print(truncated_nursing_df.shape[0])
truncated_nursing_df[['Provider Name', 'performance_id']]

In [ ]:
# Show that record 407 of the truncated table and row 16 of the performance table have the
# same values for the columns of performance table (list_of_performance_columns_df) and
# have the same performance_id (16) associated with them.
print(truncated_nursing_df[list_of_performance_columns_df].iloc[407])
print(f"performance_id for truncated is: {truncated_nursing_df['performance_id'].iloc[407]}")
performance_table_df.iloc[16]

In [ ]:
# Show the shape of truncated_nursing_df before the save point to compare
# with its shape after the save point
truncated_nursing_df.shape

In [ ]:
# Create a saving point (for the loading point in the cell below)
truncated_nursing_df.to_csv('./truncated_nursing_df2.csv', index=False)
truncated_nursing_df.to_json('./truncated_nursing_df2_by_record.json', orient='records')
truncated_nursing_df.to_json('./truncated_nursing_df2.json')


In [ ]:
# Create a loading point to skip all the previous cells when rerunning
# the code further past this cell
truncated_nursing_df = pd.read_csv('./truncated_nursing_df2.csv')

In [ ]:
# Verify that the load of the data was successful
print(truncated_nursing_df.shape)
truncated_nursing_df.head()

In [ ]:
# Create the a business table
list_of_business_columns = ['Federal Provider Number', 'Provider Name', 
    'Provider City', 'Provider Zip Code', 'Provider County Name',
    'Ownership Type', 'Number of Certified Beds', 'Number of Residents in Certified Beds', 
    'Provider Type', 'Provider Resides in Hospital',  
    'Automatic Sprinkler Systems in All Required Areas', 'Location', 'Processing Date', 
    'Latitude','Longitude', 'Adjusted Total Nurse Staffing Hours per Resident per Day',
    'performance_id']
business_table_df = pd.DataFrame()
business_table_df[list_of_business_columns] = truncated_nursing_df[list_of_business_columns] 

# Verify that the business table was created
print(business_table_df.shape)
business_table_df.head()

In [ ]:
# Save business table and create a saving point (for the loading point in the cell below)
business_table_df.to_csv('../TableData/business_table.csv', index=False)
business_table_df.to_json('../TableData/business_table.json')

In [ ]:
# Create a loading point to skip all the previous cells when rerunning
# the code further past this cell
business_table_df = pd.read_csv('../TableData/business_table.csv')

In [ ]:
# Verify that the load of the data was successful
print(business_table_df.shape)
business_table_df.head()

In [ ]:
# Create a zipcode table
list_of_state_zipcodes = ['Provider State', 'Provider Zip Code']
zip_state_table_df = pd.DataFrame()
zip_state_table_df[list_of_state_zipcodes] = truncated_nursing_df[list_of_state_zipcodes]

# Verify that the load of the data was successful
print(zip_state_table_df.shape)
zip_state_table_df.head()

In [ ]:
# Show number of rows before dropping the duplicates
print(zip_state_table_df.shape[0])

In [ ]:
# Drop duplicate rows
zip_state_table_df.drop_duplicates(inplace=True)

In [ ]:
# Verify that rows were dropped
print(zip_state_table_df.shape)
zip_state_table_df.head()

In [ ]:
# Save zipcode table and create a saving point (for the loading point in the cell below)
zip_state_table_df.to_csv('../TableData/zipcode_table.csv', index=False)
zip_state_table_df.to_json('../TableData/zipcode_table.json')

In [ ]:
# Create a loading point to skip all the previous cells when rerunning
# the code further past this cell
zip_state_table_df = pd.read_csv('../TableData/zipcode_table.csv')

In [ ]:
# Verify that the load of the data was successful
print(zip_state_table_df.shape)
zip_state_table_df.head()